In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
t=Tokenizer()
text="""Deep learning is part of a broader family of machine learning methods based"""

In [7]:
t.fit_on_texts([text])
print("단어 집합 : ",t.word_index)

단어 집합 :  {'learning': 1, 'of': 2, 'deep': 3, 'is': 4, 'part': 5, 'a': 6, 'broader': 7, 'family': 8, 'machine': 9, 'methods': 10, 'based': 11}


케라스는 텍스트 데이터를 처리하는 신셩망에 사용할 수 있는 Embedding 레이어를 제공
* Embedding 레이어의 입력 데이터는 정수 인코딩되어서, 각 단어가 고유한 정수로 표현
e=Embedding(input_dim,output_dim,input_length=100)
* input dim : 이것은 텍스트 데이터의 어휘 크기이다. 예를 들어 데이터가 0~9사이의 값으로 정수 인코딩된 경우 어휘의 크기는 10이 된다.
* output_dim : 이것은 단어가 표현되는 벡터 공간의 크기이다. 각 단어에 대해 이 레이어의 출력 벡터 크기를 정의한다. 예를 들어 32 또는 100이상일 수 잇다.
* input_length : 입력 시퀀스의 길이이다. 예를 들어 모든 입력 문서가 100개의 단어로 구성되어 있으면 100이 된다.
* 입력 형태 : 2D텐서(batch_size, sequence_length)의 형태이다. 정수 인코딩 형태이어야 한다. 즉 정수의 시퀀스이어야 한다.
* 출력 형태 : 3D 텐서(batch_size, sequence_length,output_dim) 형태이다.

In [8]:
seq=t.texts_to_sequences([text])[0]
print(text,"->",seq)

Deep learning is part of a broader family of machine learning methods based -> [3, 1, 4, 5, 2, 6, 7, 8, 2, 9, 1, 10, 11]


In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X=pad_sequences([[7,8,9],[1,2,3,4,5],[7]],maxlen=3,padding='pre')
print(X)

[[7 8 9]
 [3 4 5]
 [0 0 7]]


# 케라스 Embedding 레이어
* 케라스는 텍스트 데이터를 처리하는 신셩망에 사용할 수 있는 Embedding 레이어를 제공
    * Embedding 레이어의 입력 데이터는 정수 인코딩되어서, 각 단어가 고유한 정수로 표현
* e = Embedding(input_dim, output_dim, input_length=100)
    * input_dim : 이것은 텍스트 데이터의 어휘 크기이다. 예를 들어 데이터가 0~9 사이의 값으로 정수 인코딩된 경우 어휘의 크기는 10이 된다.
    * output_dim : 이것은 단어가 표현되는 벡터 공간의 크기이다. 예를 들어 32 또는 100이상일 수 있다.
    * input_length : 입력 시퀀스의 길이이다. 예를 들어 모든 입력 문서가 100개의 단어로 구성되어 있으면 100이 된다.
    * 입력 형태 : 2D 텐서(batch_size, sequence_length)의 형태이다. 정수 인코딩 형태이어야 한다. 즉 정수의 시퀀스이어야 한다.
    * 출력 형태 : 3D 텐서(batch_size, sequence_length, output_dim)의 형태이다.

In [12]:
import numpy as np
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential

# 입력 형태 : (batch_size, input_length) = (32,3)
# 출력 형태 : (None, 3, 4)
model = Sequential()
model.add(Embedding(100,4,input_length=3))

input_array = np.random.randint(100,size=(32,3))
model.compile('rmsprop','mse')
output_array=model.predict(input_array)
print('출력 = ', output_array.shape)

1/1 [==============================] - 0s 216ms/step
출력 =  (32, 3, 4)


# 스팸 메일 분류하기

In [18]:
import numpy as np
from tensorflow.keras.layers import Embedding,Flatten,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences


docs= ['additional income',
'best price',
'big bucks',
'cash bonus',
'earn extra cash',
'spring savings certificate',
'valero gas marketing',
'all domestic employees',
'nominations for oct',
'confirmation from spinner']

labels = np.array([1,1,1,1,1,0,0,0,0,0])

vocab_size = 50
encoded_docs = [one_hot(d,vocab_size) for d in docs]
print(encoded_docs)

[[1, 5], [23, 26], [17, 43], [12, 13], [10, 42, 12], [29, 7, 4], [36, 25, 43], [30, 37, 1], [9, 24, 27], [20, 48, 15]]


In [19]:
max_length = 4
padded_docs = pad_sequences(encoded_docs,max_length,padding='post')
print(padded_docs)
print('shape = ',padded_docs.shape)

[[ 1  5  0  0]
 [23 26  0  0]
 [17 43  0  0]
 [12 13  0  0]
 [10 42 12  0]
 [29  7  4  0]
 [36 25 43  0]
 [30 37  1  0]
 [ 9 24 27  0]
 [20 48 15  0]]
shape =  (10, 4)


In [20]:
model = Sequential()
model.add(Embedding(vocab_size,8,input_length=max_length))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(padded_docs,labels,epochs=50,verbose=0)
loss,accuracy = model.evaluate(padded_docs,labels,verbose=0)
print('정확도=',accuracy)

정확도= 1.0


In [27]:
test_doc=['cash income']
encoded_docs = [one_hot(d,vocab_size)for d in test_doc]
padded_docs=pad_sequences(encoded_docs,maxlen=max_length,padding='post')

print(model.predict(padded_docs))

1/1 [==============================] - 0s 16ms/step
[[0.55819374]]
